In [2]:
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import reduce
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [3]:
caracteristics = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/caracteristics.csv',encoding='latin-1', sep=",") 
places = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/places.csv', sep=",")
users = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/users.csv', sep=",")
vehicules = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/vehicles.csv', sep=",")

C:\Users\mnhelal\AppData\Local\Temp\ipykernel_984\3722665817.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristics = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/caracteristics.csv',encoding='latin-1', sep=",")
C:\Users\mnhelal\AppData\Local\Temp\ipykernel_984\3722665817.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  places = pd.read_csv('C:/Users/mnhelal/OneDrive - Université Paris 1 Panthéon-Sorbonne/Bureau/Paris/Algorithmique et Python/archive/places.csv', sep=",")


In [4]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [5]:
places = places[places.nbv <= 10]
places['prof'] = places['prof'].replace(0.0, 1.0)

places['plan'] = places['plan'].replace(0.0, 1.0)


places['surf'] = places['surf'].replace(0.0, 1.0)



places['circ'] = places['circ'].replace(0.0, 2.0)


C:\Users\mnhelal\AppData\Local\Temp\ipykernel_984\1603870997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places['prof'] = places['prof'].replace(0.0, 1.0)
C:\Users\mnhelal\AppData\Local\Temp\ipykernel_984\1603870997.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places['plan'] = places['plan'].replace(0.0, 1.0)
C:\Users\mnhelal\AppData\Local\Temp\ipykernel_984\1603870997.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [6]:
domtom = [971,972,973,974,976,975] #But this doesn't include les Antilles (Martinique or Guadeloupe) cause they're not departements

caracteristics = caracteristics[~caracteristics.dep.isin(domtom)]


In [7]:
caracteristics['lum'] = caracteristics['lum'].replace(0, 1)

caracteristics['int'] = caracteristics['int'].replace(0, 1)

In [8]:
# Dropping les Antilles so we can stay on only le Métropole
caracteristics = caracteristics[caracteristics.gps != "A"]
caracteristics

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,5.0,"46, rue Sonneville",M,0.0,0.0,590
1,201600000002,16,3,16,1800,1,2,6,1.0,6.0,5.0,1a rue du cimetière,M,0.0,0.0,590
2,201600000003,16,7,13,1900,1,1,1,1.0,6.0,11.0,NaN,M,0.0,0.0,590
3,201600000004,16,8,15,1930,2,2,1,7.0,3.0,477.0,52 rue victor hugo,M,0.0,0.0,590
4,201600000005,16,12,23,1100,1,2,3,1.0,3.0,11.0,rue Joliot curie,M,0.0,0.0,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837479,200500084521,5,12,30,2000,5,1,1,1.0,2.0,55.0,A7-Sens Marseille vers L,NaN,NaN,NaN,130
837480,200500084522,5,12,31,1230,1,1,1,1.0,6.0,71.0,A51-Sens Aix en Provence,NaN,NaN,NaN,130
837481,200500084523,5,12,31,1412,1,1,1,8.0,7.0,55.0,A7-Sens Marseille vers L,NaN,NaN,NaN,130
837482,200500084524,5,12,31,2020,3,1,1,2.0,7.0,55.0,A55-Sens Martigues vers,NaN,NaN,NaN,130


In [9]:
#le merge 
from functools import reduce
Merge1 = [caracteristics, places]
Merge_1 = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc'],how='left'),Merge1)


Merge2 = [vehicules, users]
Merge_2 = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc','num_veh'],how='left'),Merge2)


Merge3 = [Merge_2, Merge_1]

accidents = reduce(lambda left,right: pd.merge(left,right,on=['Num_Acc'],how='left'),Merge3)

MemoryError: Unable to allocate 100. MiB for an array with shape (7, 1876005) and data type float64

In [ ]:
missing_values_table(accidents)

In [ ]:
accidents = accidents.drop(['v1', 'v2', 'pr', 'pr1', 'lartpc', 'adr', 'voie','long','lat','gps','locp', 'actp', 'etatp','larrout','env1','vosp','place','manv','obsm','obs','senc'], axis=1)

In [ ]:
for i in accidents.columns:
    accidents[i].fillna(accidents[i].mode()[0], inplace=True)
    print(accidents)

In [ ]:
missing_values_table(accidents)

In [ ]:
accidents.info()

## One Hot Encoding

In [ ]:
df_ohe = accidents.drop(['jour','hrmn','com','num_veh','dep','an_nais','catv','occutc','agg', 'grav'], axis = 1)
df_ohe = df_ohe.astype(float)
df_ohe

In [ ]:
y = accidents['grav']
y

In [ ]:
df_ohe.shape

In [ ]:
selected_columns = ['an', 'mois', 'lum', 'int', 'atm', 'col', 'catr', 'circ', 'nbv', 'prof',
       'plan', 'surf', 'infra', 'situ', 'catu', 'sexe', 'trajet',
       'secu', 'choc']


accidents_ohe = pd.get_dummies(df_ohe, columns =selected_columns)
accidents_ohe = accidents_ohe.drop(['Num_Acc'], axis = 1)
accidents_ohe

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(accidents_ohe)
data_rescaled

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
reduced = pca.fit_transform(data_rescaled)

In [ ]:
reduced.shape

In [ ]:
pd.DataFrame(reduced)